In [1]:
import json
import os
import math
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision.models as tm
import torchvision.transforms.functional as tf
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torch.utils.data import DataLoader
from PIL import Image
from tqdm import tqdm
%matplotlib inline
sys.path.append('../code')
sys.path.append('../../nn_tools/')

In [2]:
from core.data_utils import read_dcms, get_spacing, read_annotation, SPINAL_DISC_ID, SPINAL_VERTEBRA_ID, rotate_point
from core.visilization import visilize_coord, visilize_distmap, visilize_annotation
from core.key_point import KeyPointAcc, KeyPointDataLoader, KeyPointModel, NullLoss, SpinalModelBase
from core.key_point import KeyPointBCELossV2, SpinalModel, KeyPointBCELoss
from core.structure import DICOM, Study, construct_studies
from core.disease import DiseaseModelBase, DiseaseModel, DisDataLoader, Evaluator
from nn_tools import torch_utils

In [3]:
train_studies, train_annotation, train_counter = construct_studies(
    '../data/lumbar_train150', '../data/lumbar_train150_annotation.json')
valid_studies, valid_annotation, valid_counter = construct_studies(
    '../data/train/', '../data/lumbar_train51_annotation.json')

  0%|          | 0/51 [00:00<?, ?it/s]

{'T11-T12': 1}


100%|##########| 51/51 [00:36<00:00,  1.40it/s]


In [4]:
# vertebra_labels, disc_labels = [], []
# for k, v in train_annotation.items():
#     vertebra_labels.append(v[0])
#     disc_labels.append(v[1])
# vertebra_labels = torch.cat(vertebra_labels)
# disc_labels = torch.cat(disc_labels)

# from collections import Counter
# vertebra_counter = Counter(vertebra_labels[:, -1].numpy().tolist())
# vertebra_counter = torch.tensor([vertebra_counter[i] for i in range(len(vertebra_counter))], dtype=torch.float)
# vertebra_weight = vertebra_counter.mean() / vertebra_counter
# print(vertebra_weight)

# disc_counter = Counter(disc_labels[:, -1].numpy().tolist())
# disc_counter = torch.tensor([disc_counter[i] for i in range(len(disc_counter))], dtype=torch.float)
# disc_weight = disc_counter.mean() / disc_counter
# print(disc_weight)

In [5]:
# {k: len(v) for k, v in train_counter.items()}

In [6]:
# {k: len(v) for k, v in valid_counter.items()}

In [7]:
train_dataloader = DisDataLoader(train_studies, train_annotation, batch_size=8, sagittal_size=[512, 512],
                                 transverse_size=[256, 256], k_nearest=1, num_workers=3, num_rep=10,
                                 prob_rotate=1, max_angel=180)

In [8]:
# for study_uid, study in train_studies.items():
#     frame = study.t2_sagittal_middle_frame
#     assert (study_uid, frame.series_uid, frame.instance_uid) in train_annotation

# for study_uid, study in valid_studies.items():
#     frame = study.t2_sagittal_middle_frame
#     assert (study_uid, frame.series_uid, frame.instance_uid) in valid_annotation

# for study_uid, study in train_studies.items():
#     if study.t2_transverse_uid is None:
#         print(study_uid)

# for data, label in tqdm(train_dataloader, ascii=True):
#     pass

In [9]:
train_images = {}
for study_uid, study in train_studies.items():
    frame = study.t2_sagittal_middle_frame
    train_images[(study_uid, frame.series_uid, frame.instance_uid)] = frame.image

In [14]:
backbone = resnet_fpn_backbone('resnet50', True)
spinal_model = SpinalModel(train_images, train_annotation,
                           num_candidates=128, num_selected_templates=8,
                           max_translation=0.05, scale_range=(0.9, 1.1), max_angel=10)
kp_model = KeyPointModel(backbone, len(SPINAL_VERTEBRA_ID), len(SPINAL_DISC_ID),
                         pixel_mean=0.5, pixel_std=1,
                         loss=KeyPointBCELossV2(lamb=1), spinal_model=spinal_model)
dis_model = DiseaseModel(
    kp_model, sagittal_size=[512, 512], loss_scaler=0.01, use_kp_loss=True, share_backbone=True
).cuda(0)
# dis_model = torch.nn.DataParallel(dis_model, device_ids=[0, 1])
dis_model

DiseaseModel(
  (backbone): KeyPointModel(
    (backbone): BackboneWithFPN(
      (body): IntermediateLayerGetter(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): FrozenBatchNorm2d()
        (relu): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): FrozenBatchNorm2d()
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): FrozenBatchNorm2d()
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn3): FrozenBatchNorm2d()
            (relu): ReLU(inplace=True)
            (downsample): Sequential(
              (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (1): FrozenBat

In [11]:
# small_studies, train_annotation, train_counter = construct_studies(
#     '../data/small/', '../data/lumbar_train150_annotation.json')
# small_dataloader = DisDataLoader(small_studies, train_annotation, batch_size=8, sagittal_size=[512, 512],
#                                  transverse_size=[256, 256], k_nearest=1, num_workers=3, num_rep=10,
#                                  prob_rotate=1, max_angel=180)
# batch_data, batch_label = next(iter(small_dataloader))

# visilize_coord(batch_data[0][0], batch_data[2][0])

# batch_pred = dis_model(*batch_data)

# tf.to_pil_image(batch_pred[1][0])

In [12]:
# study = Study('../data/lumbar_train150/study10/')
# pred_coord, *_ = dis_model.eval().module(study)
# human_coord = study.t2_sagittal_middle_frame.pixel_coord2human_coord(pred_coord.cpu())
# human_coord
# study.t2_transverse.point_distance(human_coord)
# [dicom.image_position for dicom in study.t2_transverse]
# study.t2_transverse.k_nearest(human_coord, 1, 8)
# k_nearest = study.t2_transverse_k_nearest(pred_coord.cpu(), 2, [256, 256], 8)
# k_nearest.shape
# tf.to_pil_image(k_nearest[8][0])

In [15]:
evaluator = Evaluator(dis_model, valid_studies, '../data/lumbar_train51_annotation.json',
                      num_rep=20, metric='macro precision')
optimizer = torch.optim.AdamW(dis_model.parameters(), lr=1e-5)
max_step = 50*len(train_dataloader)
fit_result = torch_utils.fit(
    dis_model,
    train_data=train_dataloader,
    valid_data=None,
    optimizer=optimizer,
    max_step=max_step,
    loss=NullLoss(),
    metrics=[NullLoss()],
    is_higher_better=True,
    evaluate_per_steps=len(train_dataloader),
    evaluate_fn=evaluator,
    checkpoint_dir='../models',
#     early_stopping=5*len(train_dataloader),
)

using NullLoss as training loss, using NullLoss(higher is better) as early stopping metric


100%|##########| 1020/1020 [00:19<00:00, 52.46it/s]


step 188 train NullLoss: 0.12798237800598145
valid macro precision: 0.3791408630852574
valid avg key point acc: 0.38993716836236664
valid disc v1 precision: 0.7026143746707108
valid disc v2 precision: 0.1021069821035014
valid disc v3 precision: 0.5695067233405057
valid disc v4 precision: 7.299269007405985e-08
valid disc v5 precision: 4.9751238830722504e-08
valid macro f1: 0.23897850934284542
valid micro f1: 0.3625218942275154
valid vertebra v1 precision: 0.42492918272355906
valid vertebra v2 precision: 0.8548286560145962


100%|##########| 1020/1020 [00:23<00:00, 43.10it/s]


step 376 train NullLoss: 0.05666559189558029
valid macro precision: 0.3888227076111525
valid avg key point acc: 0.37096570374289467
valid disc v1 precision: 0.674714099753136
valid disc v2 precision: 0.20650096724661696
valid disc v3 precision: 0.42926829613325385
valid disc v4 precision: 0.09615389792898744
valid disc v5 precision: 5.208332790798668e-08
valid macro f1: 0.23915900348055724
valid micro f1: 0.3163265344709312
valid vertebra v1 precision: 0.562674091215928
valid vertebra v2 precision: 0.7524475489168175


...

step 5640 train NullLoss: 0.00947966706007719
valid macro precision: 0.4853740707070182
valid avg key point acc: 0.8476638457972105
valid disc v1 precision: 0.7117969117790253
valid disc v2 precision: 0.5728301875799217
valid disc v3 precision: 0.29831933196808125
valid disc v4 precision: 0.4072727340165284
valid disc v5 precision: 0.082191795130418
valid macro f1: 0.5534521454247128
valid micro f1: 0.7177142822308572
valid vertebra v1 precision: 0.4348508654677907
valid vertebra v2 precision: 0.8903566690073624


100%|##########| 1020/1020 [00:18<00:00, 54.48it/s]


step 5828 train NullLoss: 0.009328084997832775
valid macro precision: 0.4642391226222441
valid avg key point acc: 0.847404708174565
valid disc v1 precision: 0.7825443759117912
valid disc v2 precision: 0.37211155582292343
valid disc v3 precision: 0.5845147200760148
valid disc v4 precision: 0.017421636416610705
valid disc v5 precision: 0.09967321569695375
valid macro f1: 0.5087340495482019
valid micro f1: 0.7135977952414938
valid vertebra v1 precision: 0.5290423852733945
valid vertebra v2 precision: 0.8643659691580206


 44%|####3     | 82/188 [00:20<00:26,  3.94it/s]


KeyboardInterrupt: 

In [16]:
dis_model.kp_model = torch.load('../models/size512_rotate1_180_AdamW_1e-5.kp_model', map_location='cuda:0')

/home/cxt/anaconda3/envs/torch15/lib/python3.8/site-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'core.key_point.model.KeyPointModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [22]:
torch.save(dis_model.cpu(), '../models/2020070102.dis_model')

In [17]:
evaluator()

100%|##########| 1020/1020 [00:36<00:00, 27.92it/s]


[('macro precision', 0.5431561827905897),
 ('avg key point acc', 0.8748156364852632),
 ('disc v1 precision', 0.7095078814189405),
 ('disc v2 precision', 0.5374617731274959),
 ('disc v3 precision', 0.33943089757254274),
 ('disc v4 precision', 0.5036231881432472),
 ('disc v5 precision', 0.16588420451066654),
 ('macro f1', 0.621341299579327),
 ('micro f1', 0.7233316229997807),
 ('vertebra v1 precision', 0.760670723760039),
 ('vertebra v2 precision', 0.7855146110011965)]

In [ ]:
# evaluator = Evaluator(dis_model, train_studies, '../data/lumbar_train150_annotation.json', num_rep=3)
# evaluator()

In [18]:
testA_studies = construct_studies('../data/lumbar_testA50/')

100%|##########| 50/50 [00:34<00:00,  1.43it/s]


In [19]:
result = []
for study in tqdm(testA_studies.values(), ascii=True):
    result.append(dis_model.eval()(study, True))

100%|##########| 50/50 [00:01<00:00, 27.12it/s]


In [ ]:
i = 9
visilize_annotation(testA_studies[result[i]['studyUid']].t2_sagittal_middle_frame.image, result[i])

In [20]:
with open('../predictions/2020070102.json', 'w') as file:
    json.dump(result, file)
    